# Reading files from the LAMDA database

In this notebook, we demonstrate how `pythonradex` can be used to conveniently read atomic data from files that follow the [LAMDA format](https://home.strw.leidenuniv.nl/~moldata/molformat.html). Such files can be download from e.g. the [EMAA](https://emaa.osug.fr/) or [LAMDA](https://home.strw.leidenuniv.nl/~moldata/) databases.

In [1]:
from pythonradex import LAMDA_file
from scipy import constants
datafilepath = 'co.dat'
data = LAMDA_file.read(datafilepath,read_frequencies=True)

The data is stored in a dictionary containing all levels, radiative transitions and collisional transitions:

In [2]:
levels = data['levels']
rad_transitions = data['radiative transitions']
coll_transitions = data['collisional transitions']

Lets first look at the levels. This is a list containing all atomic energy levels (in the form of instances of the `Level` class, see the API for details) listed in the file. It is ordered the same way as in the file. Let’s access the statistical weight and energy of the 3rd level as an example (note that the index is zero–based):

In [3]:
level_index = 2
print(f'checking level with index {level_index}')
print(f'g = {levels[level_index].g}')
print(f'E = {levels[level_index].E} J')

checking level with index 2
g = 5.0
E = 2.2913493923384677e-22 J


Similarly, the radiative transitions are stored in a list, also ordered as they appear in the file. Each element of the list is an instance of the `RadiativeTransition` class (see the API for details). Let’s see how many radiative transitions there are:

In [4]:
len(rad_transitions)

40

Let’s look at the 2-1 transition of CO. This is the second transition listed in the file, so its index is 1.

In [5]:
CO_21 = rad_transitions[1]

We can access the upper and lower level of the transition. These are instance of the `Level` class:

In [6]:
print(f'statistical weight of upper level: {CO_21.up.g}')
print(f'energy of the lower level: {CO_21.low.E} J')

statistical weight of upper level: 5.0
energy of the lower level: 7.637950693853056e-23 J


Let’s look at some of the other attributes of this transition such as frequency, energy difference and Einstein coefficients. For a complete list of the available attributes, please see the API.

In [7]:
print(f'rest frequency: {CO_21.nu0/constants.giga} GHz')
print(f'energy of the transition: {CO_21.Delta_E} J')
print(f'Einstein A21: {CO_21.A21}')

rest frequency: 230.538 GHz
energy of the transition: 1.527554322953162e-22 J
Einstein A21: 6.91e-07


We can also compute the excitation temperature of the transition for given fractional populations of the lower and upper level (x1 and x2):

In [8]:
CO_21.Tex(x1=0.3,x2=0.1)

array(6.87446893)

Finally, let’s have a look at the collisional transitions. This is a dictionary containing the transitions for each collision partner. Let’s see which collision partners are present:

In [9]:
coll_transitions.keys()

dict_keys(['para-H2', 'ortho-H2'])

Let’s look at collisions with ortho-H2. This is a list with instances of the `CollisionalTransition` class (see API). How many collisional transitions are there?

In [10]:
len(coll_transitions['ortho-H2'])

820

Similarly to the radiative transition, there are a number of attributes we can access. Let’s look at a randomly chosen transition:

In [11]:
coll_trans = coll_transitions['ortho-H2'][99]
print(f'g = {coll_trans.low.g}')
print(f'DeltaE = {coll_trans.Delta_E} J')

g = 17.0
DeltaE = 5.2654882500940636e-21 J


Again, see the API to get all attributes. Like for radiative transitions, one can calculate the excitation temperature. In addition, one can get the collisional transition rates. The LAMDA data file provides these rates at specific temperatures. We can request an interpolated rate at any temperature within the limits defined in the file:

In [12]:
K12,K21 = coll_trans.coeffs(Tkin=100.5)
print(f'K12 = {K12} m3/s')
print(f'K21 = {K21} m3/s')

K12 = 2.482103904202611e-19 m3/s
K21 = 6.4704300000000014e-18 m3/s


Numpy arrays are also allowed as input:

In [13]:
import numpy as np
Tkin = np.array((52.3,70.4,100.2,150.4))
K12,K21 = coll_trans.coeffs(Tkin=Tkin)
print(f'K12 [m3/s] = {K12}')
print(f'K21 [m3/s] = {K21}')

K12 [m3/s] = [6.88755045e-21 4.64677588e-20 2.45259938e-19 9.61028767e-19]
K21 [m3/s] = [5.929830e-18 6.136280e-18 6.466572e-18 7.112992e-18]
